# A nonlinear oscillator

## Références

- A. Der Kiureghian, M. De Stefano, Efficient algorithm for second-order reliability analysis, J. Eng. Mech. 117 (12) (1991) 2904–2923.
- J.-M. Bourinet, F. Deheeger, M. Lemaire, Assessing small failure probabilities by combined subset simulation and Support Vector Machines, Struct. Saf. 33 (6) (2011) 343–353.
- J.-M. Bourinet, Rare-event probability estimation with adaptive support vector regression surrogates, Reliab. Eng. Syst. Saf. 150 (2016) 210–221.
- V. Dubourg, Adaptive surrogate models for reliability analysis and reliability-based design optimization, PhD thesis, Université Blaise Pascal – Clermont II, 2011.
- Vincent Chabridon, Mathieu Balesdent, Jean-Marc Bourinet, Jérôme Morio, Nicolas Gayton. Evaluation of failure probability under parameter epistemic uncertainty: application to aerospace system reliability assessment. Aerospace Science and Technology, Elsevier, 2017, 69, pp.526-537.
- Analyse de sensibilité fiabiliste avec prise en
compte d’incertitudes sur le modèle probabiliste, Thèse présentée par Vincent Chabridon, 2019.

## Description

The aim is to assess reliability of a two-degree-of-freedom primary-secondary system under a white noise base acceleration. 

<img src="nonlinear-oscillator.png" width="400" />

The basic variables characterizing the physical behavior are 
* the masses $m_p$ and $m_s$, 
- spring stiffnesses $k_p$ and $k_s$, 
- natural frequencies $\omega_p$ and $\omega_s$,
- damping ratios $\xi_p$ and $\xi_s$, 

where the subscripts p and s respectively refer to the primary and secondary oscillators.

The reliability of the system can be evaluated using the following limit state function:
$$
g(X) = F_s - 
3 k_s \sqrt{\frac{\pi S_0}{4\xi_s \omega_s^3}
\frac{\xi_a\xi_s}{\xi_p\xi_s(4\xi_a^2 + r^2) + \gamma \xi_a^2}
\frac{(\xi_p\omega_p^3 + \xi_s\omega_s^3)\omega_p}{4\xi_a\omega_a^4}
}
$$
where 
- $F_s$ : the force capacity of the secondary spring,
- $S_0$ is the intensity of the white noise, 
- $\omega_p = (k_p/m_p)^{1/2}$,
- $\omega_s = (k_s/m_s)^{1/2}$, 
- $\omega_a = (\omega_p + \omega_s)/2$ the average frequency ratio, 
- $\gamma =m_s/m_p$ the mass ratio, 
- $\xi_a = (\xi_p + \xi_s)/2$ the average damping ratio and 
- $r = (\omega_p − \omega_s)/\omega_a$ a tuning parameter.

| Random var. | Distribution | Mean | Coef. of var. |
|--|--|--|--|
| $F_s$ | Lognormale | 21.5 | 10% |
| $m_p$ | Lognormale | 1.5 | 10% |
| $m_s$ | Lognormale | 0.01 | 10% |
| $k_p$ | Lognormale | 1 | 20% |
| $k_s$ | Lognormale | 0.01 | 20% |
| $\xi_p$ | Lognormale | 0.05 | 40% |
| $\xi_s$ | Lognormale | 0.02 | 50% |
| $S_0$ | Lognormale | 100 | 10% |

The two interesting characteristics of this application test-case are its set of non-normal basic random variables and the fact that it suffers from a highly nonlinear limit-state surface (which prevents from using any FORM-based approach). Moreover, following, it seems relevant to consider the mean of the force capacity $F_s$ as the most influent distribution parameter on the failure probability. 
$$
pf_{ref} = 4.75 \times 10^{-5}.
$$

In [1]:
import numpy as np
import openturns as ot

In [2]:
def oscillator(x):
    fs, mp, ms, kp, ks, xip, xis, S0 = x
    omegap = np.sqrt(kp/mp)
    omegas = np.sqrt(ks/ms)
    omegaa = 0.5*(omegap+omegas)
    gamma = ms/mp
    xi_a = 0.5*(xip+xis)
    theta = 1./omegaa*(omegap-omegas)
    F = fs - 3*ks*np.sqrt(np.pi*S0/(4.*xis*omegas**3)*
        xi_a*xis/(xip*xis*(4.*xi_a**2+theta**2)+gamma*xi_a**2)*
        (xip*omegap**3+xis*omegas**3)*omegap/(4.*xi_a*omegaa**4))
    return [F]

In [3]:
dim = 8
g = ot.PythonFunction(dim,1,oscillator)

In [4]:
mean_list = [21.5, 1.5, 0.01, 1., 0.01, 0.05, 0.02, 100.]
cov_list = [0.1, 0.1, 0.1, 0.2, 0.2, 0.4, 0.5, 0.1]
myCollection = ot.DistributionCollection(dim)
for i, (mu, cov) in enumerate(zip(mean_list, cov_list)):
    myParam = ot.LogNormalMuSigma(mu, mu*cov, 0.)
    myCollection[i] = ot.ParametrizedDistribution(myParam)

In [5]:
distribution = ot.ComposedDistribution(myCollection)
X = ot.RandomVector(distribution)

In [6]:
Y = ot.CompositeRandomVector(g, X)

In [7]:
myEvent = ot.Event(Y, ot.LessOrEqual(), 0.0)

In [8]:
blocksize = 1000
experiment = ot.MonteCarloExperiment()
myAlgo = ot.ProbabilitySimulationAlgorithm(myEvent, experiment)
myAlgo.setMaximumOuterSampling(10000)
myAlgo.setBlockSize(blocksize)
myAlgo.setMaximumCoefficientOfVariation(0.1)

In [9]:
myAlgo.run()

In [10]:
result = myAlgo.getResult()

In [11]:
outersampling = result.getOuterSampling()
outersampling

2218

In [12]:
funccalls = blocksize * outersampling
print("Number of function calls = %d" % (funccalls))

Number of function calls = 2218000


In [13]:
pf = result.getProbabilityEstimate()
pf

4.553651938683502e-05

In [14]:
alpha = 0.05
pflen = result.getConfidenceLength(1-alpha)
print("%.2f%% confidence interval = [%f,%f]" % ((1-alpha)*100,pf-pflen/2,pf+pflen/2))

95.00% confidence interval = [0.000037,0.000054]
